In [ ]:
import pandas as pd

def read_tsv_to_dataframe(file_path, selected_columns=None):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    header = lines[0].strip().split('\t')

    data = []

    for line in lines[3:]:
        fields = line.strip().split('\t')
        data.append(fields[:211])

    df = pd.DataFrame(data, columns=header)
    
    if selected_columns!=None:
        df = df[selected_columns]
        
    return df

def ECdata(Towers=['K67', 'K77', 'K83']):
    file_head = 'CD32_Fluxes_Brazil_1842/data/'
    file_tail = 'day_CfluxBF.txt'
    selected_columns = ['Year_LBAMIP', 'DoY_LBAMIP', 'Hour_LBAMIP', 'NEEnogap_5day_sco2_ust']
    data_dict = {}

    for tower in Towers:
        file_path = file_head + tower + file_tail
        df = read_tsv_to_dataframe(file_path, selected_columns)
        df.rename(columns={'Year_LBAMIP': 'Year', 'DoY_LBAMIP': 'Julian_Day', 'Hour_LBAMIP': 'Hour', 'NEEnogap_5day_sco2_ust': 'NEE'}, inplace=True)
        data_dict[tower] = df
    
    return data_dict

dict_ECdata = ECdata()

In [ ]:
import pandas as pd

def read_tsv_to_dataframe(file_path, selected_columns=None):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    header = lines[0].strip().split('\t')

    data = []

    for line in lines[3:]:
        fields = line.strip().split('\t')
        data.append(fields[:211])

    df = pd.DataFrame(data, columns=header)
    
    if selected_columns is not None:
        df = df[selected_columns]
        
    # Convert Julian Day and Year to date
    df['Year'] = df['Year_LBAMIP'].astype(float).astype(int)
    df['Julian_Day'] = df['DoY_LBAMIP'].astype(float).astype(int)
    df['date'] = pd.to_datetime(df['Year'], format='%Y') + pd.to_timedelta(df['Julian_Day'] - 1, unit='D')
    
    return df

def ECdata(Towers=['K67', 'K77', 'K83']):
    file_head = 'CD32_Fluxes_Brazil_1842/data/'
    file_tail = 'day_CfluxBF.txt'
    selected_columns = ['Year_LBAMIP', 'DoY_LBAMIP', 'Hour_LBAMIP', 'NEEnogap_5day_sco2_ust']
    data_dict = {}

    for tower in Towers:
        file_path = file_head + tower + file_tail
        df = read_tsv_to_dataframe(file_path, selected_columns)
        df.rename(columns={'Hour_LBAMIP': 'Hour', 'NEEnogap_5day_sco2_ust': 'NEE'}, inplace=True)
        data_dict[tower] = df
    
    return data_dict

dict_ECdata = ECdata()


In [ ]:
import ee
ee.Initialize()

lst = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')

i_date = '2000-01-01'

f_date = '2007-01-01'

scale = 500

lst = lst.select('SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7').filterDate(i_date, f_date)

k67_poi = ee.Geometry.Point(-54.959, -2.857)

lst_k67_poi = lst.getRegion(k67_poi, scale).getInfo()

k77_poi = ee.Geometry.Point(-54.8885, -3.0202)

lst_k77_poi = lst.getRegion(k77_poi, scale).getInfo()

k83_poi = ee.Geometry.Point(-54.9707, -3.017)

lst_k83_poi = lst.getRegion(k83_poi, scale).getInfo()

In [ ]:
lst_k67 = pd.DataFrame(lst_k67_poi[1:], columns=lst_k67_poi[0])
lst_k67

In [ ]:
import pandas as pd
from datetime import datetime

# Assuming lst_k67 is already defined as the DataFrame

def add_year_and_julian_day(df):
    df['date_str'] = df['id'].str.split('_').str[-1]
    df['date'] = pd.to_datetime(df['date_str'], format='%Y%m%d')
    df = df.drop(columns=['date_str'])
    
    return df

# Applying the function to lst_k67
lst_k67 = add_year_and_julian_day(lst_k67)

# Display the new DataFrame
lst_k67


In [ ]:
ec = dict_ECdata['K67']
sat = lst_k67

merged = pd.merge(ec, sat, on='date', suffixes=('_ec', '_sat'), how='inner')

merged

In [61]:
import pandas as pd
import ee
ee.Initialize()

def read_tsv_to_dataframe(file_path, selected_columns=None):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    header = lines[0].strip().split('\t')

    data = []

    for line in lines[3:]:
        fields = line.strip().split('\t')
        data.append(fields[:211])

    df = pd.DataFrame(data, columns=header)
    
    if selected_columns is not None:
        df = df[selected_columns]
        
    # Convert Julian Day and Year to date
    df['Year'] = df['Year_LBAMIP'].astype(float).astype(int)
    df['Julian_Day'] = df['DoY_LBAMIP'].astype(float).astype(int)
    df['date'] = pd.to_datetime(df['Year'], format='%Y') + pd.to_timedelta(df['Julian_Day'] - 1, unit='D')
    
    return df

def combined_data(Towers=['K67', 'K77', 'K83']):
    file_head = 'CD32_Fluxes_Brazil_1842/data/'
    file_tail = 'day_CfluxBF.txt'
    selected_columns = ['Year_LBAMIP', 'DoY_LBAMIP', 'Hour_LBAMIP', 'NEEnogap_5day_sco2_ust']
    data_dict = {}
    lst = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
    i_date = '2000-01-01'
    f_date = '2007-01-01'
    scale = 500
    lst = lst.select('SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7').filterDate(i_date, f_date)
    
    def add_year_and_julian_day(df):
        df['date_str'] = df['id'].str.split('_').str[-1]
        df['date'] = pd.to_datetime(df['date_str'], format='%Y%m%d')
        df = df.drop(columns=['date_str'])
    
        return df

    for tower in Towers:
        if tower == 'K67':
            poi = ee.Geometry.Point(-54.959, -2.857)
        elif tower == 'K77':
            poi = ee.Geometry.Point(-54.8885, -3.0202)
        elif tower == 'K83':
            poi = ee.Geometry.Point(-54.9707, -3.017)
        file_path = file_head + tower + file_tail
        ec_df = read_tsv_to_dataframe(file_path, selected_columns)
        ec_df.rename(columns={'Hour_LBAMIP': 'Hour', 'NEEnogap_5day_sco2_ust': 'NEE'}, inplace=True)
        
        lst_poi = lst.getRegion(poi, scale).getInfo()
        sat_df = pd.DataFrame(lst_poi[1:], columns=lst_poi[0])
        sat_df = add_year_and_julian_day(sat_df)        
        
        df = pd.merge(ec_df, sat_df, on='date', how='inner')
        
        data_dict[tower] = df
    
    return data_dict

data_dict = combined_data()

In [65]:
def prep_data(data_dict):
    x_train = []
    y_train = []
    for key, df in data_dict.items():
        for index, row in df.iterrows():
            y_train.append(row['NEE'])
            sr = [row['SR_B1'], row['SR_B2'], row['SR_B3'], row['SR_B4'], row['SR_B5'], row['SR_B7']]
            x_train.append(sr)
    
    return x_train, y_train

x_train, y_train = prep_data(data_dict)


[[19636, 18769, 18615, 23841, 18752, 15988],
 [8535, 9239, 8756, 19638, 12951, 9669],
 [58368, 46288, 49729, 26256, 18932, 15188],
 [13948, 14266, 14096, 19198, 15835, 14436],
 [15040, 15324, 15030, 21400, 13088, 11335],
 [8104, 8340, 8160, 16733, 11735, 9225],
 [9816, 10346, 9921, 21309, 14633, 10940],
 [52871, 44967, 46939, 28407, 20359, 15374],
 [16797, 15510, 15885, 22705, 18139, 14851],
 [7858, 8369, 7959, 17133, 12153, 9001],
 [9099, 9471, 9018, 18333, 13028, 9864],
 [20924, 19664, 19736, 22778, 18521, 15296],
 [14335, 14530, 14291, 19607, 14154, 12100],
 [11795, 11857, 11688, 17409, 14219, 12109],
 [15397, 13662, 14709, 17957, 15416, 12244],
 [8149, 8894, 8291, 20408, 12914, 9454],
 [27454, 17415, 21880, 23958, 18302, 15636],
 [17428, 13751, 15047, 18171, 13185, 10960],
 [8492, 9140, 8633, 20010, 12566, 9484],
 [8469, 8858, 8451, 17000, 11386, 9111],
 [8293, 8785, 8131, 21013, 12935, 9320],
 [63399, 39176, 51619, 26977, 20658, 16029],
 [65535, 63104, 65535, 29580, 24833, 21919],